In [52]:
import pymongo

# Verbindung zur MongoDB-Instanz herstellen
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Datenbank auswählen
db = client["comunio"]

# Collection auswählen
raw_player_collection = db["raw_player"]

In [54]:
# Erstelle eine Aggregation, um die Dokumente nach "Playerid" zu gruppieren und das "Datum" und "Marktwert" für jedes Dokument zu extrahieren
pipeline = [
  {"$group": {
    "_id": "$Playerid",
    "dates": {"$push": "$Datum"},
    "market_values": {"$push": "$Marktwert"}
  }}
]

# Führe die Aggregation aus und speichere das Ergebnis in einer Variablen
aggregate_result = raw_player_collection.aggregate(pipeline)

# Wähle die Sammlung "player_mw" aus, in die du die zusammengefassten Daten schreiben möchtest
player_mw_collection = db["player_mw"]


# Verwende eine Schleife, um die Ergebnisse der Aggregation zu durchlaufen und die Daten in die Sammlung zu schreiben
for result in aggregate_result:
  player_id = result["_id"]
  dates = result["dates"]
  market_values = result["market_values"]
  
  player_mw_collection.update_one({"Playerid": player_id}, {"$set": {"Dates": dates, "MarketValues": market_values}}, upsert=True)